In [41]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

In [211]:
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b
print("Base point:\n", P)
print("p :", p)
print("a :", a)
print("b :", b)
print("n :", n)


Base point:
 (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798 , 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
p : 115792089237316195423570985008687907853269984665640564039457584007908834671663
a : 0
b : 7
n : 115792089237316195423570985008687907852837564279074904382605163141518161494337


In [212]:
API_URL = 'http://10.92.52.175:5000/'

stuID =  25139  ## Change this to your ID number
stuID = 26846
#server's Identitiy public key
IKey_Ser = Point(93223115898197558905062012489877327981787036929201444813217704012422483432813 , 8985629203225767185464920094198364255740987346743912071843303975587695337619, E)

#Send Public Identitiy Key Coordinates and corresponding signature
def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)		
    if((response.ok) == False): print(response.json())

#Send the verification code
def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    print(response.json())

#Send SPK Coordinates and corresponding signature
def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)		
    if((response.ok) == False): 
        print(response.json())
    else: 
        res = response.json()
        return res['SPKPUB.X'], res['SPKPUB.Y'], res['H'], res['S']

#Send OTK Coordinates and corresponding hmac
def OTKReg(keyID,x,y,hmac):
    mes = {'ID':stuID, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Send the reset code to delete your Identitiy Key
#Reset Code is sent when you first registered
def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Sign your ID  number and send the signature to delete your SPK
def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Send the reset code to delete your Identitiy Key
def ResetOTK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)		
    if((response.ok) == False): print(response.json())

#Pseudo-client will send you 5 messages to your inbox via server when you call this function
def PseudoSendMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "PseudoSendMsg"), json = mes)		
    print(response.json())

#get your messages. server will send 1 message from your inbox 
def ReqMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)	
    print(response.json())	
    if((response.ok) == True): 
        res = response.json()
        return res["IDB"], res["OTKID"], res["MSGID"], res["MSG"], res["EK.X"], res["EK.Y"]

#If you decrypted the message, send back the plaintext for grading
def Checker(stuID, stuIDB, msgID, decmsg):
    mes = {'IDA':stuID, 'IDB':stuIDB, 'MSGID': msgID, 'DECMSG': decmsg}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)		
    print(response.json())

def Status(stuID, h, s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "Status"), json = mes)	
    print(response.json())
    if (response.ok == True):
        res = response.json()
        return res['numMSG'], res['numOTK'], res['StatusMSG']	

In [126]:
# IK data
sA = 75365099691681163466259435055151388598990196366045037773536006665067396329561
Q_x = 87058090705547232336742506266448902457032608537577399368869745072849975094263
Q_y = 73863361865121729253370141189521784002483174318560123943873444021233722393171

s_ik = 38435919104545028465405229083851199426577612833328471419512121279897034838259
h_ik = 40139398399024429188254655216662123008175195428486006903289784284292094948763

# SPK data
spk_private = 49760804016305672242628858452671561631883295017037579045439043666340119331411
spk_pub_x = 13279412479756039023006939294151267045839717479408817839293710814463378950452
spk_pub_y = 93617390663867776116902322986952603938589221775532373175859757173961357675363

h_spk = 70079813329073930856804606363140448062677695844504871159148958480169809247078
s_spk = 42702578452500864554658062321887469197282606023250231586097905168842648683915

# OTK data
otk_private = [82643481877244209249664745771237418234477690096430811959717657855485886045638, 41174952876732969873061234384234992214813764471036650196683240643681961363261, 5583145587846578787702511470815367992841936970155580384792772696588865599734, 10972659824334756040423367058252295538222729497376669524883637366213216276543, 24476832630378983018284218279520822890885956931179901813178437213634407485252, 97006687499945984175088601268948450846237282033793682470458998922800746644169, 74160389491792105526360889492360032639008866468092295253480592634938957625351, 2064776794757669561866729584104676040173421819183532961139782611452732109747, 58659695490434911403509798003920364225859338325533233728677532945968361081857, 19148444294741063747898074518353049744162854913178507691342149982373162093145]

In [270]:
# IK data (ekin)
stuID = 26846
sA = 26037317628054985808039282432726226222027022467068096985918786230345691957027
Q_x = 34966638871069348443434840366429984004050324904300954986624560986102166574831
Q_y = 18659130989987264139111259616350811871077628350857638087893010137535414968071

s_ik = 10711401428822947968805605574831727095538167654217243597237904326250472141120
h_ik = 2310173870477934111899724596823579375648537282076021725587322962459991319407

# SPK data
spk_private = 108276252172224876587334255735165101721975917778097283393900202750918785959710
spk_pub_x = 97710353570137819193379937963704253670763857898842070763814037579480279837106
spk_pub_y = 23341606178871725188404066641975812905580291605280961572149830211055662093589

h_spk = 50104479202124469309008236680532873175969290556870197285989963025491089808550
s_spk = 56354363435458804487927205429170920223354636346201640247065025929905617989041

# OTK data
otk_private = [17659173398494903595859837776265486632267282615973692643366636303927303998573,
 14473739146864278029042056324411178141706024431675113027712939417954206283188,
 107116472759777461710331535470627755119415732096838955840726928786766866378666,
 85691346369690975217694199379623033734339376768201202924941100869866832183492,
 87710805958830227451133596519053378261563154085891809842886507991110103767176,
 68151747642013147387751098033133766425559394070275313886866434982699598255656,
 76462017522914954423485840189321925592045212928343396294807271130065994199717,
 57854208037700907960848908966134841900785197104320808743787041225192523991242,
 82293262707090954663575897700430467053294325183793165143798892711835364326076,
 75356985148119232958632457566701480858646842676511666125346107634457844697652]

In [81]:
# Signing my stuID with my private IK in order to get messages from the server:
k = random.randint(1, n-2)
R = k * P
r = R.x % n

r_byte_array = r.to_bytes((r.bit_length() +7)//8, byteorder = 'big')
m_byte_array = stuID.to_bytes((stuID.bit_length() +7)//8, byteorder = 'big')
r_m = r_byte_array + m_byte_array #concatenation of r and m where m is stuID

h = SHA3_256.new(r_m)
h = int.from_bytes(h.digest(), byteorder='big') % n

s = (k - (sA * h)) % n
print("s:", s)
print("h:", h)

PseudoSendMsg(h,s)

s: 17315019618322541976448592766302620815363715653266534858153261252238475693602
h: 106294522765085930420359036741991921998940543032722415350027706092632704734613


In [271]:
Status(stuID, h_ik, s_ik)

Sending message is:  {'ID': 26846, 'H': 2310173870477934111899724596823579375648537282076021725587322962459991319407, 'S': 10711401428822947968805605574831727095538167654217243597237904326250472141120}
{'numMSG': 0, 'numOTK': 10, 'StatusMSG': 'You have 0 unread messages in your mailbox.\n You have 10 OTKs left. The largest key id is 9'}


(0,
 10,
 'You have 0 unread messages in your mailbox.\n You have 10 OTKs left. The largest key id is 9')

In [272]:
# requesting messages from the server:
otkID_array = []
msgID_array = []
msg_array = []
for i in range(5):
    stuIDB, otkID, msgID, msg, ek_x, ek_y = ReqMsg(h,s)
    msg_array.append(msg)
    otkID_array.append(otkID)
    msgID_array.append(msgID)
    

Sending message is:  {'ID': 26846, 'H': 106294522765085930420359036741991921998940543032722415350027706092632704734613, 'S': 17315019618322541976448592766302620815363715653266534858153261252238475693602}
There is no message in your message box. Pseudo client will send you messages if you call this method: PseudoSendMsg


TypeError: cannot unpack non-iterable NoneType object

In [269]:
first_iteration = True
counter = 0
for message in msg_array:
    message_byte_array = message.to_bytes((message.bit_length() +7)//8, byteorder = 'big')
    
    # hmac of the message
    message_HMAC = message_byte_array[len(message_byte_array)-32:]
    #print(message_HMAC) 

    # nonce and the ciphertext
    message_with_nonce = message_byte_array[:len(message_byte_array)-32]
    #print(message_with_nonce)

    # ciphertext
    ciphertext = message_byte_array[8:len(message_byte_array)-32]
    #print(ciphertext)


    # _session key generation start_
    EK_B_point = Point(ek_x, ek_y, curve = E)
    T = otk_private[otkID] * EK_B_point
    T_x = T.x
    T_y = T.y

    T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
    T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
    U = T_x_byte_array + T_y_byte_array + b'MadMadWorld'


    K_S = SHA3_256.new(U)
    # _session key generation end_

    # _key derivation start_
    if(first_iteration):
        K_KDF = K_S
        first_iteration = False
    else:
        K_KDF = SHA3_256.new(K_HMAC.digest() + b'YouWillNotHaveTheDrink')

    K_ENC = SHA3_256.new(K_KDF.digest() + b'LeaveMeAlone')
    K_HMAC = SHA3_256.new(K_ENC.digest() + b'GlovesAndSteeringWheel')
    # _key derivation end_



    # obtaining hmac from ciphertext
    hmac = HMAC.new(K_HMAC.digest(), ciphertext, digestmod=SHA256)
    hmac = hmac.digest()
    
    # checking MAC values
    if(hmac == message_HMAC):
        print("HMAC is verified")

        # dencryption of the message
        cipher = AES.new(K_ENC.digest(), AES.MODE_CTR, nonce = message_with_nonce[0:8])
        dtext = cipher.decrypt(ciphertext)
        decrypted_message = dtext.decode('utf-8')

        print("Decrypted message:", decrypted_message)

        # cheking decrypted message
        #Checker(25139, stuIDB, msgID_array[counter], decrypted_message)
    else: 
        print("HMAC is not verified")
        #Checker(25139, stuIDB, msgID_array[counter], 'INVALIDHMAC')

    counter = counter + 1


    print("_________________________________")
    

HMAC is not verified
_________________________________
HMAC is not verified
_________________________________
HMAC is not verified
_________________________________
HMAC is not verified
_________________________________
HMAC is not verified
_________________________________
